# Load Data
Bilder einlesen und Dateinamen anpassen

In [ ]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset

import warnings


In [ ]:
import pandas as pd # Deals with data

In [ ]:
excel_readings = pd.read_excel('../data/raw/moroweg_strom_gas.xlsx')
excel_readings.head()

In [ ]:
manual_readings = excel_readings.iloc[:,[0,1,3,5]]
manual_readings = manual_readings.melt(id_vars=["Date", "Kommentar"], 
        var_name="Meter Type", 
        value_name="Value")
manual_readings[["Meter Type", "Unit"]] = manual_readings['Meter Type'].str.split(' ',expand=True)
manual_readings = manual_readings[["Date", "Meter Type", "Unit", "Value", "Kommentar"]]
manual_readings

In [ ]:
## Code Snippet for exifread for a sample image
import exifread
import os

#path_name = os.path.join(os.pardir, 'data', 'raw', '2017', '2017-03-03 15.06.47.jpg')
path_name = "..\data\processed\gas\IMG_20200405_173910.jpg"

# Open image file for reading (binary mode)
f = open(path_name, 'rb')

# Return Exif tags
tags = exifread.process_file(f)

# Show Tags
for tag in tags.keys():
    if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
        print("Key: %s, value %s" % (tag, tags[tag]))

In [ ]:
def extract_file_meta(file_path):
    basename = os.path.basename(file_path)
    
    # Open image file for reading (binary mode)
    f = open(file_path, 'rb')

    # Read EXIF
    tags = exifread.process_file(f)
    
    try:
        exif_datetime = str(tags["EXIF DateTimeOriginal"])
    except KeyError:
        warnings.warn("File {file_path} does not appear to have a date in EXIF Tags.".format(file_path=file_path))
        return()
        #exif_datetime = "2020:01:01 00:00:00"
            
    # Format Date
    datetime = pd.to_datetime(exif_datetime, format = "%Y:%m:%d %H:%M:%S")
    date = pd.to_datetime(datetime.date())
    
    return(basename, datetime, date, file_path)

In [ ]:
def meta_from_files(files):
    files_meta = []
    for file_path in files:
        files_meta.append(extract_file_meta(file_path))
    df = pd.DataFrame.from_records(files_meta, columns = ("Filename", "Datetime", "Date", "Filepath"))
    return(df)

In [ ]:
def meta_from_dir(dir_path):
    files = [top + os.sep + f for top, dirs, files in os.walk(dir_path) for f in files]
    files_meta = meta_from_files(files)
    return(files_meta)

In [ ]:
gas_dir = os.path.join(os.pardir, "data", "processed", "gas")
gas_files_meta = meta_from_dir(gas_dir)

In [ ]:
gas_files_meta

In [ ]:
strom_dir = os.path.join(os.pardir, "data", "processed", "strom")
strom_files_meta = meta_from_dir(strom_dir)

In [ ]:
strom_files_meta

## Add Flag if Picture has been Labelled

In [ ]:
gas_label_dir = os.path.join(os.pardir, "data", "labelled", "gas", "vott-json-export")
gas_labelled_files = [os.path.basename(f) for top, dirs, files in os.walk(gas_label_dir) for f in files]
gas_files_meta["Labelled"] = gas_files_meta.apply(lambda row: True if row["Filename"] in gas_labelled_files else False, axis=1)
gas_files_meta

In [ ]:
strom_label_dir = os.path.join(os.pardir, "data", "labelled", "strom", "vott-json-export")
strom_labelled_files = [os.path.basename(f) for top, dirs, files in os.walk(strom_label_dir) for f in files]
strom_files_meta["Labelled"] = strom_files_meta.apply(lambda row: True if row["Filename"] in strom_labelled_files else False, axis=1)
strom_files_meta

## Join Picture Data with Manual Readings

### Strom

In [ ]:
manual_readings.head(2)

In [ ]:
strom_readings_manual = manual_readings[manual_readings["Meter Type"] == "Strom"]
strom_readings_manual.head(2)

In [ ]:
strom_files_meta.head(2)

In [ ]:
strom = strom_files_meta.merge(strom_readings_manual, left_on="Date", right_on="Date")
strom

### Gas

In [ ]:
manual_readings.head(2)

In [ ]:
gas_readings_manual = manual_readings[manual_readings["Meter Type"] == "Gas"]
gas_readings_manual.head(2)

In [ ]:
gas_files_meta.head(2)

In [ ]:
gas = gas_files_meta.merge(gas_readings_manual, left_on="Date", right_on="Date")
gas

## Return one dataframe in the end

In [ ]:
dataset = pd.concat([strom, gas])
dataset

In [ ]:
dataset.to_csv("../data/processed/dataset.csv")